In [1]:
from deep_emotion_recognition_trained import DeepEmotionRecognizer
# initialize instance
# inherited from emotion_recognition.EmotionRecognizer
# default parameters (LSTM: 128x2, Dense:128x2)
network = DeepEmotionRecognizer(emotions=['angry', 'happy', 'neutral','sad'], n_rnn_layers=2, n_dense_layers=2, rnn_units=128, dense_units=128)

# get the accuracy
#print(deeprec.test_score())
# predict angry audio sample
#prediction = deeprec.predict('data/validation/Actor_10/03-02-05-02-02-02-10_angry.wav')
#print(f"Prediction: {prediction}")

[TESS&RAVDESS] There are 809 training audio files for category:angry
[TESS&RAVDESS] There are 148 testing audio files for category:angry
[TESS&RAVDESS] There are 806 training audio files for category:happy
[TESS&RAVDESS] There are 148 testing audio files for category:happy
[TESS&RAVDESS] There are 586 training audio files for category:neutral
[TESS&RAVDESS] There are 94 testing audio files for category:neutral
[TESS&RAVDESS] There are 813 training audio files for category:sad
[TESS&RAVDESS] There are 147 testing audio files for category:sad
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 339
[EMO-DB] Training samples: 271
[EMO-DB] Testing samples: 67
[+] Writed EMO-DB CSV File
[Custom Dataset] There are 48 training audio files for category:happy
[Custom Dataset] There are 23 testing audio files for category:happy
[Custom Dataset] There are 49 training audio files for category:neutral
[Custom Dataset] There are 33 testing audio files for category:neutral
[Custom Dat

In [41]:
print(network.predict_proba("data/validation/Actor_10/03-02-02-01-02-02-10_calm.wav"))

{'angry': 0.012038359, 'happy': 0.1863404, 'neutral': 0.022269364, 'sad': 0.77935183}


In [2]:
import os

def delete_folder_files(dir):

    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

In [3]:
### need os imported ###
#import os

def predict_emotions(dir):
    predictions = []
    for f in os.listdir(dir):
        predictions.append(network.predict_proba(dir + '/' +f))
        
        #print(network.predict_proba(dir + '/' +f))
        #print(dir + '/' + f)
        
    return predictions

In [4]:
import numpy as np
import pyaudio
import wave
from array import array
import struct
import time


def record(rate = 24414, chunk = 512, seconds = 5):
    RATE = rate
    CHUNK = chunk
    RECORD_SECONDS = seconds
    FORMAT = pyaudio.paInt32
    CHANNELS = 1
    
    
    
    
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
    
    #Inizialize a non-silent signal array 
    data = array('h', np.random.randint(size = 512, low = 0, high = 500))
    
    #SESSION START
    print("** session started")
    total_predictions = [] # A list for all predictions in the session.
    tic = time.perf_counter()
    
    #while is_silent(data) == False:
    for i in range(0,3):
        print("* recording...")
        frames = [] 
        data = np.nan # Reset 'data' variable.

        timesteps = int(RATE / CHUNK * RECORD_SECONDS) # => 339

        # Insert frames to 'output.wav'.
        for k in range(0, timesteps):
            data = array('l', stream.read(CHUNK)) 
            frames.append(data)
            
            output_file = "recordings/output" + str(i) + ".wav"
            wf = wave.open(output_file, 'wb')
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(p.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(frames))

        print("* done recording")
        
    print("** session ended")
    global is_recording
    is_recording = False


In [5]:
delete_folder_files('recordings')

record()

** session started
* recording...
* done recording
* recording...
* done recording
* recording...
* done recording
** session ended


In [45]:
predictions = predict_emotions('recordings')

print(predictions)

[{'angry': 0.0011650968, 'happy': 0.09741572, 'neutral': 0.85697293, 'sad': 0.044446323}, {'angry': 0.00199729, 'happy': 0.25454012, 'neutral': 0.6790977, 'sad': 0.064364895}, {'angry': 0.1961456, 'happy': 0.2292252, 'neutral': 0.14974439, 'sad': 0.42488477}]


In [5]:
def emotion_average(results):
    #instatiate averages vector
    average_pred = {}
    for emo in results[0]:
        average_pred[emo] = 0
    
    length = len(results)
    emo_num = len(results[0])
    for emotion in results[0]:
        for idx in range(0,length):
            average_pred[emotion] += results[idx][emotion]
        
    for emotion in average_pred:
        average_pred[emotion] /= length
        
    return average_pred

In [47]:
pred = emotion_average(predictions)
print(pred)

{'angry': 0.06643599361026038, 'happy': 0.19372701396544775, 'neutral': 0.5619383454322815, 'sad': 0.1778986615439256}


In [48]:
#print(network.predict_proba('brombeisMONO.wav'))

In [6]:
import keyboard as kb
is_recording = False

while True:
    if kb.is_pressed('space'):
        if not is_recording:
            is_recording = True
            delete_folder_files('recordings')
            record()
            #print('A pressed \n')
    elif kb.is_pressed('Esc'): 
        break

** session started
* recording...
* done recording
* recording...
* done recording
* recording...
* done recording
** session ended
** session started
* recording...
* done recording
* recording...
* done recording
* recording...
* done recording
** session ended
